In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat

In [3]:
SXR_horizontal_r = np.loadtxt('data/Line_of_Sight/detector_0_x.txt') / 1000.0  # shape (Nh, 2)
SXR_horizontal_z = np.loadtxt('data/Line_of_Sight/detector_0_z.txt') / 1000.0  # from mm to m

SXR_vertical_r   = np.loadtxt('data/Line_of_Sight/detector_1_x.txt') / 1000.0  # shape (Nv, 2)
SXR_vertical_z   = np.loadtxt('data/Line_of_Sight/detector_1_z.txt') / 1000.0  # from mm to m

SXR_oblique_r    = np.loadtxt('data/Line_of_Sight/detector_2_x.txt') / 1000.0  # shape (No, 2)
SXR_oblique_z    = np.loadtxt('data/Line_of_Sight/detector_2_z.txt') / 1000.0  # from mm to m

Nh, Nv, No = len(SXR_horizontal_r), len(SXR_vertical_r), len(SXR_oblique_r)
M = Nh + Nv + No

# ---- Build unified arrays: starts and end per LOS ----
start_r = np.concatenate([SXR_horizontal_r[:, 0], SXR_vertical_r[:, 0], SXR_oblique_r[:, 0]])
start_z = np.concatenate([SXR_horizontal_z[:, 0], SXR_vertical_z[:, 0], SXR_oblique_z[:, 0]])
end_r   = np.concatenate([SXR_horizontal_r[:, 1], SXR_vertical_r[:, 1], SXR_oblique_r[:, 1]])
end_z   = np.concatenate([SXR_horizontal_z[:, 1], SXR_vertical_z[:, 1], SXR_oblique_z[:, 1]])

# ---- Geometry (vectorized) ----
dr = end_r - start_r
dz = end_z - start_z

# avoid division by zero for verticals:
is_vertical = np.isclose(dr, 0.0, atol=1e-12)
end_r[ np.where(is_vertical==True) ] += 0.0001 ##### ASK IF THIS IS OKAY
dr = end_r - start_r

m = dz/dr
m[np.where(np.isclose(m, 0.0, atol=1e-12)==True)] += 1e-6
b = start_z - m*start_r

In [4]:
# Load the data, for a specific time, and interpolation grid 
t = 5; shot = 58089
equilibrium_data = loadmat("data/imas_equilibrium_58089.mat")

r = equilibrium_data['ids']['interp2D'][0][0]['r'][0][0].T # shape (N=60,N) | a grid
z = equilibrium_data['ids']['interp2D'][0][0]['z'][0][0].T # shape (N=60,N) | a grid

border_left, border_right = r[0][0], r[0][-1] # grid horizontal delimitation
border_bottom, border_top = z[0][0], z[-1][0] # grid vertical delimitation

In [5]:
SXR_horizontal_r.T[1] = np.array([border_left]*Nh)
SXR_vertical_z.T[1] = np.array([border_bottom]*Nv)
SXR_oblique_z.T[1]  = np.array([border_bottom]*No)

SXR_horizontal_z.T[1] = m[:Nh]*border_left + b[:Nh]
SXR_vertical_r.T[1] = (border_bottom -  b[Nh:-No])/m[Nh:-No]
SXR_oblique_r.T[1] = (border_bottom - b[Nh+Nv:])/m[Nh+Nv:]

In [19]:
SXR_horizontal = pd.concat(
    [pd.DataFrame(SXR_horizontal_r,columns=["start_r","end_r"]),
    pd.DataFrame(SXR_horizontal_z,columns=["start_z","end_z"]),
    pd.DataFrame(m[:Nh],columns=["m"]),
    pd.DataFrame(b[:Nh],columns=["b"])],
    axis=1
)#.to_csv("data/Line_of_Sight/horizontal_sxr.csv",index=False)

In [18]:
SXR_vertical = pd.concat(
    [pd.DataFrame(SXR_vertical_r,columns=["start_r","end_r"]),
    pd.DataFrame(SXR_vertical_z,columns=["start_z","end_z"]),
    pd.DataFrame(m[Nh:-No],columns=["m"]),
    pd.DataFrame(b[Nh:-No],columns=["b"])],
    axis=1
)#.to_csv("data/Line_of_Sight/oblique_sxr.csv",index=False)

In [17]:
SXR_oblique = pd.concat(
    [pd.DataFrame(SXR_oblique_r,columns=["start_r","end_r"]),
    pd.DataFrame(SXR_oblique_z,columns=["start_z","end_z"]),
    pd.DataFrame(m[Nh+Nv:],columns=["m"]),
    pd.DataFrame(b[Nh+Nv:],columns=["b"])],
    axis=1
)#.to_csv("data/Line_of_Sight/vertical_sxr.csv",index=False)